<a href="https://colab.research.google.com/github/vmotta/Algebra-Linear-Python/blob/master/geracao_pb_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==1.4.1   

     |████████████████████████████████| 41.2MB 810kB/s 
     |████████████████████████████████| 1.7MB 35.7MB/s 
     |████████████████████████████████| 890kB 40.1MB/s 


In [2]:
#Importar o Tensorflow
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from google.protobuf import text_format

#criação dos diretórios para salvar os checkpoints e o .pb
save_dir = './checkpoints/'
save_pb = './pb/'
save_log = './log/'


        
# Para limpar as definições de variáveis e operações do grafo do tensorflow
tf.reset_default_graph()          

#criação da base de dados
x=np.array([[0,0],[0,1],[1,0],[1,1]])#atributos previsores
y=np.array([[0],[1],[1],[0]])#atributis da classe

#definição das camadas
neuronios_entrada = 2
neuronios_oculta = 3
neuronios_saida = 1

#criação dos pesos das camadas de forma aleatória
w = {'oculta' : tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta]), name='w_oculta'),
    'saida': tf.Variable(tf.random_normal([neuronios_oculta,neuronios_saida]), name='w_saida')}

#criação dos bias das camadas de forma aleatória
b = {'oculta' : tf.Variable(tf.random_normal([neuronios_oculta]), name='b_oculta'),
    'saida': tf.Variable(tf.random_normal([neuronios_saida]), name='b_saida')}

#criação dos placesholders
xph = tf.placeholder(tf.float32,[4, neuronios_entrada], name='xph')
yph = tf.placeholder(tf.float32,[4, neuronios_saida], name='yph')

#formulas
camada_oculta = tf.add(tf.matmul(xph,w['oculta']),b['oculta'])
camada_oculta_ativacao = tf.sigmoid(camada_oculta)
camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']),b['saida'])
camada_saida_ativacao = tf.sigmoid(camada_saida)
erro = tf.losses.mean_squared_error(yph, camada_saida_ativacao)
otimizador = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(erro)


#cria os diretórios caso não existam
if not os.path.exists(save_dir):
    os.makedirs(save_dir)  
if not os.path.exists(save_pb):
    os.makedirs(save_pb)
    
#cria inicialização para iniciar variáveis do tensorflow dentro da sessão 
init = tf.global_variables_initializer()

#inicia sessão tensorflow
with tf.Session() as sess:
    #inicia as variáveis do tensorflow
    sess.run(init)
    #inicia o contador de épocas
    epocas=0
    #inicia o erro médio com valor alto para entrar no while
    erro_medio = 999
    #roda a rede até o erro for pequeno 
    while erro_medio >= 1e-2:
        #inicia o erro médio com 
        erro_medio = 0
        #inicia o feed dos placeholders e roda o otmizador e retorna o erro
        _, custo =sess.run([otimizador, erro], feed_dict = {xph: x, yph: y})
        #a cada 200 epocas ou o erro médio ser zero calcula o erro médio
        if (epocas % 200 == 0) or (erro_medio == 0):
            #calcula o erro médio para cada saída
            erro_medio += custo/4
            #incrementa as épocas
            epocas+=1    
            #print('Epoca: ', epocas, ' -> Erro Médio: ', erro_medio)
    #salva os pesos finais e o bias final        
    w_final, b_final = sess.run([w,b]) 
    
#formulas teste de rede
camada_oculta_teste = tf.add(tf.matmul(xph,w_final['oculta']),b_final['oculta'])
camada_oculta_ativacao_teste = tf.sigmoid(camada_oculta_teste)
camada_saida_teste = tf.add(tf.matmul(camada_oculta_ativacao_teste, w_final['saida']),b_final['saida'])
camada_saida_ativacao_teste = tf.sigmoid(camada_saida_teste)
#print(camada_saida_ativacao_teste)

feed_dict_test = {xph: x}
with tf.Session() as sess:
    sess.run(init)
    #execução da rede de teste
    yph = sess.run(camada_saida_ativacao_teste, feed_dict=feed_dict_test)
    print(yph)
    
    writer = tf.summary.FileWriter(save_pb, sess.graph)

    #nome do arquivo de salvamento de log e faz a saida em formato .pbtxt
    filename = 'my_test_model'
   
    # Exporta o checkpoint para SavedModel
    #salva o grafo e eventos da sessão em arquivos de log 
    train_writer = tf.summary.FileWriter(save_pb, sess.graph)
    #exporta para formato .pb
    tf.train.write_graph(tf.GraphDef(), save_pb,filename+'.pb', as_text=False)
    test_summary_writer = tf.summary.FileWriter(save_log, sess.graph) 
    

    
    

W0705 15:16:14.055882 139890394916736 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[[0.12523778]
 [0.75896513]
 [0.83248574]
 [0.24093248]]
